# Train

In [1]:
#!pip install dominate ; pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
import time
time.sleep(0)

In [ ]:
!cd ../.. ; \
python src/models/Pix2pixHD/train.py \
--ngf 64 --ndf 32 --nef 16 --niter 120 \
--augmentation_aggressive --niter_decay 120 \
--checkpoints_dir temp/checkpoint/Pix2pixHD/ \
--name conditional_binary_agg --dataroot xBD \
--dataset_mode physics_aligned_bin --n_blocks_local 4 \
--no_instance --label_nc 0 --input_nc 4 --batchSize 7 --gpu_ids 0,1,2,3,4,5,7

------------ Options -------------
augmentation_aggressive: True
batchSize: 7
beta1: 0.5
checkpoints_dir: temp/checkpoint/Pix2pixHD/
continue_train: False
data_type: 32
dataroot: xBD
dataset_mode: physics_aligned_bin
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0, 1, 2, 3, 4, 5, 7]
input_nc: 4
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 4
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: conditional_binary_agg
ndf: 32
nef: 16
netG: global
ngf: 64
niter: 120
niter_decay: 120
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_cr

In [3]:
#Generate predictions over the test set. 
!cd ../.. ; python src/models/Pix2pixHD/test.py --checkpoints_dir temp/checkpoint/Pix2pixHD/  --ngf 64 --nef 16 --netG local --phase test --how_many 2000 --name conditional_binary_agg --dataroot xBD --dataset_mode physics_aligned_bin --no_instance --label_nc 0 --n_blocks_global 8 --input_nc 4 --gpu_ids 1,2,3,4,5,6,7 --batchSize 7 --results_dir temp/Pix2pixHD/


------------ Options -------------
aspect_ratio: 1.0
augmentation_aggressive: False
batchSize: 7
checkpoints_dir: temp/checkpoint/Pix2pixHD/
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: xBD
dataset_mode: physics_aligned_bin
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [1, 2, 3, 4, 5, 6, 7]
how_many: 2000
input_nc: 4
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 8
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: conditional_binary_agg
nef: 16
netG: local
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: scale_width
results_dir: temp/Pix2pixHD/
serial_batches: False
slosh_inference: False
tf_log: False
use_dropout: False
use_encoded_image: F

### We move things around at the moment in ./Temp/
The model output is placed into ./temp/ .We use this directory to work, then we carefully selects what goes into ./results/.
./temp/ does not get uploaded into our bucket, only ./results/ do!

In [4]:
# Move synthesized results to a folder to be used by the segmentation network
!cd ../../temp/Pix2pixHD/conditional_binary_agg/test_latest/ ; mkdir to_segment; cp -R images/*synthesized*.jpg to_segment

In [5]:
#Create Segmentation Masks
#Generate masks for all of the data generated by pix2pixHD_baseline
! cd ../.. ; python src/models/Pix2pix-CycleGAN/test.py --phase to_segment --dataroot temp/Pix2pixHD/conditional_binary_agg/test_latest/ --direction AtoB --results_dir ./temp/Pix2pixHD/conditional_binary_agg_masks --dataset_mode single --model pix2pix --checkpoints_dir ./pretrained/Pix2pix-CycleGAN/flood_segmentation/scratch_1024_plus/ --name . --num_test 2000 --no_flip --gpu_ids 0 --max_dataset_size 2000 --batch_size 1  --load_size 1024 --crop_size 1024


----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./pretrained/Pix2pix-CycleGAN/flood_segmentation/scratch_1024_plus/	[default: ./checkpoints]
                crop_size: 1024                          	[default: 256]
                 dataroot: temp/Pix2pixHD/conditional_binary_agg/test_latest/	[default: None]
             dataset_mode: single                        	[default: aligned]
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
              

### Now we copy our generated images into the right ./Results/. directory

In [6]:
! cd ../.. ; mkdir results/Pix2pixHD/conditional_binary_agg/ ; cp -R temp/Pix2pixHD/conditional_binary_agg/test_latest/images/* results/Pix2pixHD/conditional_binary_agg/ ; cp -R temp/Pix2pixHD/conditional_binary_agg_masks/to_segment_latest/images/*image_fake_B.png results/Pix2pixHD/conditional_binary_agg/

mkdir: cannot create directory ‘results/Pix2pixHD/conditional_binary_agg/’: File exists
